# Acquire

1st step is to aquire the data from the Codeup db.

In [114]:
#Libraries

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# preparing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, explained_variance_score

# modeling and evaluating
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.linear_model import TweedieRegressor
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures

# Exploring
import scipy.stats as stats
import math

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
# pd.options.display.float_format = '{:20,.2f}'.format

# import acquire
# import summarize
# import prepare

In [115]:
import acquire

In [116]:
import os
import env

In [4]:
# Add how handling nulls and document how we're handling them.
# Handle outliers and document (add to README).

In [117]:
# Importing and displaying the zillow dataframe

df = acquire.get_zillow_data()
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,3.5,4.0,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,NaN,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13,0,0.025595,14297519,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,1.0,2.0,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13,1,0.055619,17052889,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13,2,0.005383,14186244,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13,3,-0.103410,12177905,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,3.0,3.0,8.0,3.0,NaN,NaN,1312.0,1312.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34185120.0,-118414640.0,278581.0,1.0,NaN,NaN,NaN,1.0,010C,LAR3,6.037124e+07,12447.0,3101.0,268548.0,96451.0,0.0,NaN,1.0,NaN,NaN,1964.0,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,NaN,NaN,6.037124e+13,4,0.006940,10887214,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [8]:
# Finding the shape of the dataframe to make sure it matches the shape of the data that I built in MySQLPro.

df.shape

(77381, 70)

In [9]:
# Using .info() to find the dtypes of the columns, and to have a baseline count of non-nulls in each column to compare to my nulls when I get into the prep stage.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 70 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77381 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77381 non-null  float64
 4   heatingorsystemtypeid         49440 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      206 non-null    float64
 7   airconditioningtypeid         24953 non-null  float64
 8   id                            77381 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77381 non-null  float64
 11  bedroomcnt                    77381 non-null  float64
 12  buildingqualitytypeid         49672 non-null  float64
 13  c

In [10]:
df.describe()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,id.1,logerror,pid
count,7.738100e+04,222.000000,50.0,77381.000000,49440.000000,15.000000,206.000000,24953.000000,7.738100e+04,50.000000,...,7.726900e+04,7.738000e+04,77381.0,7.737900e+04,77376.000000,2886.000000,7.713700e+04,77381.000000,77381.000000,7.738100e+04
mean,1.300715e+07,6.040541,7.0,261.826340,3.920510,3.933333,7.388350,1.813289,1.495139e+06,679.720000,...,1.893145e+05,4.901345e+05,2016.0,3.010954e+05,5995.532346,14.087318,6.049673e+13,38848.692754,0.016625,1.300715e+07
std,3.481346e+06,0.557285,0.0,5.141199,3.592779,0.258199,2.734542,2.967894,8.609071e+05,689.703546,...,2.300874e+05,6.534440e+05,0.0,4.925960e+05,7622.844153,2.185663,1.535242e+12,22402.429065,0.170191,3.481346e+06
min,1.071186e+07,4.000000,7.0,31.000000,1.000000,3.000000,2.000000,1.000000,3.490000e+02,38.000000,...,4.400000e+01,1.000000e+03,2016.0,1.610000e+02,19.920000,3.000000,6.037101e+13,0.000000,-4.655420,1.071186e+07
25%,1.153830e+07,6.000000,7.0,261.000000,2.000000,4.000000,7.000000,1.000000,7.520700e+05,273.000000,...,8.426500e+04,2.070000e+05,2016.0,8.550400e+04,2715.632500,14.000000,6.037311e+13,19461.000000,-0.024377,1.153830e+07
50%,1.253157e+07,6.000000,7.0,261.000000,2.000000,4.000000,7.000000,1.000000,1.497932e+06,515.000000,...,1.364990e+05,3.589755e+05,2016.0,2.033720e+05,4450.695000,15.000000,6.037603e+13,38870.000000,0.006627,1.253157e+07
75%,1.421183e+07,6.000000,7.0,266.000000,7.000000,4.000000,7.000000,1.000000,2.240535e+06,796.500000,...,2.187870e+05,5.690015e+05,2016.0,3.667965e+05,6927.790000,15.000000,6.059042e+13,58253.000000,0.039203,1.421183e+07
max,1.676893e+08,13.000000,7.0,275.000000,24.000000,4.000000,21.000000,13.000000,2.982274e+06,3560.000000,...,1.142179e+07,4.906124e+07,2016.0,4.895220e+07,586639.300000,99.000000,4.830301e+14,77613.000000,5.262999,1.676893e+08


In [12]:
df.taxamount.value_counts()

2418.22     8
3720.52     7
2217.31     5
1725.20     5
3815.22     5
           ..
5434.38     1
3716.17     1
7358.73     1
4654.70     1
10240.00    1
Name: taxamount, Length: 73825, dtype: int64

In [19]:
# I think a better way of dividing up these variables might be using cut??
def roundup(x):
    return int(math.ceil(x / 100.0)) * 100

In [25]:
def rounddown(x):
    return int(math.floor(x / 100.0)) * 100

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [6]:
df.isna().sum()

parcelid                      0
typeconstructiontypeid    77159
storytypeid               77331
propertylandusetypeid         0
heatingorsystemtypeid     27941
                          ...  
buildingclassdesc         77366
heatingorsystemdesc       27941
propertylandusedesc           0
storydesc                 77331
typeconstructiondesc      77159
Length: 70, dtype: int64

In [7]:
df.shape

(77381, 70)

In [8]:
num_row_missing = df.isna().sum()
num_row_missing

parcelid                      0
typeconstructiontypeid    77159
storytypeid               77331
propertylandusetypeid         0
heatingorsystemtypeid     27941
                          ...  
buildingclassdesc         77366
heatingorsystemdesc       27941
propertylandusedesc           0
storydesc                 77331
typeconstructiondesc      77159
Length: 70, dtype: int64

In [15]:
# Decided to round this column, but can remove the round() command if needed to match the curriculum exactly.
pct_rows_missing = round(num_row_missing/df.shape[0], 3)

In [16]:
df_missing = pd.DataFrame({'num_row_missing': num_row_missing, 'pct_rows_missing': pct_rows_missing})
df_missing.head()

,num_row_missing,pct_rows_missing
parcelid,0,0.000
typeconstructiontypeid,77159,0.997
storytypeid,77331,0.999
propertylandusetypeid,0,0.000
heatingorsystemtypeid,27941,0.361


In [17]:
# Now to create the function:

def zillow_missing_values(df):
    
    total_rows = df.shape[0]
    
    # Count of missing values per column
    num_row_missing = df.isna().sum()
    
    # Pct of missing values per column
    pct_rows_missing = num_row_missing/total_rows
    
    df_missing = pd.DataFrame({'num_row_missing': num_row_missing, 'pct_rows_missing': pct_rows_missing})
    
    return df_missing
    

In [18]:
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,logerror,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,0.025595,14297519,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,0.055619,17052889,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,0.005383,14186244,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,-0.103410,12177905,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,0.006940,10887214,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [20]:
df_missing_test = zillow_missing_values(df)
df_missing_test

,num_row_missing,pct_rows_missing
parcelid,0,0.000000
typeconstructiontypeid,77159,0.997131
storytypeid,77331,0.999354
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,27941,0.361083
...,...,...
buildingclassdesc,77366,0.999806
heatingorsystemdesc,27941,0.361083
propertylandusedesc,0,0.000000
storydesc,77331,0.999354


#### Takeaways on Missing Values

- There are three points about taking care of the missing values:
    1. Columns at > 50% missing values should probably be dropped.
    2. Or if there is no need for the column from a modeling perspective, and most of the column is missing values, then it's dropped.
    3. Conversely, if a column has > 50% missing values but there is a modeling need for that data, the column may be retained.

### Columns Missing Info

4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [40]:
df.nunique()

parcelid                        77381
typeconstructiontypeid              4
storytypeid                         1
propertylandusetypeid              13
heatingorsystemtypeid              10
buildingclasstypeid                 2
architecturalstyletypeid            5
airconditioningtypeid               5
id                              77381
basementsqft                       43
bathroomcnt                        22
bedroomcnt                         16
buildingqualitytypeid              12
calculatedbathnbr                  21
decktypeid                          1
finishedfloor1squarefeet         1787
calculatedfinishedsquarefeet     4972
finishedsquarefeet12             4868
finishedsquarefeet13               13
finishedsquarefeet15             1724
finishedsquarefeet50             1807
finishedsquarefeet6               350
fips                                3
fireplacecnt                        5
fullbathcnt                        13
garagecarcnt                       14
garagetotals

In [55]:
# Percentage of each row that have missing column info.
df.isna().sum(axis = 1)

0        36
1        33
2        34
3        32
4        29
         ..
77376    33
77377    32
77378    32
77379    34
77380    34
Length: 77381, dtype: int64

In [57]:
# Percentage of each row that do not have missing column info.

# df.notna().sum(axis = 1) / df.shape[1]
df.isna().sum(axis = 1) / df.shape[1]

0        0.514286
1        0.471429
2        0.485714
3        0.457143
4        0.414286
           ...   
77376    0.471429
77377    0.457143
77378    0.457143
77379    0.485714
77380    0.485714
Length: 77381, dtype: float64

In [107]:
# of rows missing

df.isna().sum(axis = 1).value_counts().sort_index()

23        2
24       13
25       24
26       65
27      316
28      455
29     5270
30     3455
31     9891
32    12579
33    14782
34    13327
35     5148
36     5775
37     3620
38     1925
39      285
40      230
41       29
42       23
43       28
44       78
45       50
46        5
47        3
48        3
dtype: int64

In [166]:
# pct cols missing
(df.isna().sum(axis = 1).value_counts(normalize = True).sort_index() * 100).reset_index()

,index,0
0,23,0.002585
1,24,0.016800
2,25,0.031015
3,26,0.084000
4,27,0.408369
5,28,0.588000
6,29,6.810457
7,30,4.464920
8,31,12.782208
9,32,16.255928


In [168]:
# Trying to reset the column so that I can get the dataframe above...

df_col = pd.DataFrame({'pct_cols_missing': (df.isna().sum(axis = 1).value_counts(normalize = True).sort_index() * 100).reset_index()})

ValueError: If using all scalar values, you must pass an index

In [158]:
df_col['num_cols_missing'] = df.isna().sum(axis = 1)
df_col

,pct_cols_missing,num_cols_missing
23,0.002585,33
24,0.016800,31
25,0.031015,33
26,0.084000,31
27,0.408369,31
28,0.588000,33
29,6.810457,34
30,4.464920,32
31,12.782208,34
32,16.255928,33


In [ ]:
# This isn't right.

In [159]:
def count_and_percent_missing_column(df):
    num_rows = df.loc[:].isnull().sum()
    num_cols_missing = df.loc[:, df.isna().any()].count()
    pct_cols_missing = round(df.loc[:, df.isna().any()].count() / len(df.index) * 100, 3)
    missing_cols_and_rows_df = pd.DataFrame({'num_cols_missing': num_cols_missing,
                                             'pct_cols_missing': pct_cols_missing,
                                             'num_rows': num_rows})
    missing_cols_and_rows_df = missing_cols_and_rows_df.fillna(0)
    missing_cols_and_rows_df['num_cols_missing'] = missing_cols_and_rows_df['num_cols_missing'].astype(int)
    return missing_cols_and_rows_df

In [173]:
df2_test = count_and_percent_missing_column(df)
df2_test.head()

,num_cols_missing,pct_cols_missing,num_rows
airconditioningdesc,24953,32.247,52428
airconditioningtypeid,24953,32.247,52428
architecturalstyledesc,206,0.266,77175
architecturalstyletypeid,206,0.266,77175
assessmentyear,0,0.000,0


In [163]:
df['num_cols_missing'] = df.isna().sum(axis = 1)
df.num_cols_missing.head()

0    36
1    33
2    34
3    32
4    29
Name: num_cols_missing, dtype: int64

In [78]:
df['pct_cols_missing'] = df.isna().sum(axis = 1) / df.shape[1]
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,...,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,num_cols_missing,pct_cols_missing
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,3.5,4.0,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,...,NaN,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13,0,0.025595,14297519,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,36,0.507042
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,1.0,2.0,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,...,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13,1,0.055619,17052889,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,33,0.464789
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,...,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13,2,0.005383,14186244,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,34,0.478873
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,...,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13,3,-0.103410,12177905,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN,32,0.450704
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,3.0,3.0,8.0,3.0,NaN,NaN,1312.0,1312.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34185120.0,-118414640.0,278581.0,1.0,NaN,NaN,NaN,...,LAR3,6.037124e+07,12447.0,3101.0,268548.0,96451.0,0.0,NaN,1.0,NaN,NaN,1964.0,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,NaN,NaN,6.037124e+13,4,0.006940,10887214,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN,29,0.408451


In [92]:
round(df.loc[:, df.isna().any()].count() / len(df.index) * 100, 3)

typeconstructiontypeid           0.287
storytypeid                      0.065
heatingorsystemtypeid           63.892
buildingclasstypeid              0.019
architecturalstyletypeid         0.266
airconditioningtypeid           32.247
basementsqft                     0.065
buildingqualitytypeid           64.191
calculatedbathnbr               99.213
decktypeid                       0.793
finishedfloor1squarefeet         7.784
calculatedfinishedsquarefeet    99.747
finishedsquarefeet12            95.306
finishedsquarefeet13             0.053
finishedsquarefeet15             3.889
finishedsquarefeet50             7.784
finishedsquarefeet6              0.499
fireplacecnt                    10.695
fullbathcnt                     99.213
garagecarcnt                    32.920
garagetotalsqft                 32.920
hashottuborspa                   1.988
lotsizesquarefeet               89.353
poolcnt                         20.854
poolsizesum                      1.120
pooltypeid10             

In [104]:
df.loc[:, df.isna().any()]

,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,basementsqft,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertyzoningdesc,regionidcity,regionidneighborhood,regionidzip,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,storydesc,typeconstructiondesc
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,633.0,NaN,4506.0,NaN,NaN,NaN,NaN,NaN,NaN,53571.0,NaN,96978.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,537569.0,11013.72,NaN,NaN,6.059063e+13,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,1.0,1.0,1.0,0.0,NaN,12647.0,NaN,NaN,NaN,NaN,NaN,NaN,13091.0,NaN,97099.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,376000.0,5672.48,NaN,NaN,6.111001e+13,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,440.0,NaN,8432.0,1.0,NaN,NaN,NaN,1.0,NaN,21412.0,NaN,97078.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,479489.0,6488.30,NaN,NaN,6.059022e+13,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2.0,NaN,NaN,NaN,NaN,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,13038.0,1.0,NaN,NaN,NaN,1.0,LCR110000*,396551.0,NaN,96330.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,36225.0,1777.51,NaN,NaN,6.037300e+13,NaN,NaN,NaN,Central,NaN,NaN
4,NaN,NaN,2.0,NaN,NaN,1.0,NaN,8.0,3.0,NaN,NaN,1312.0,1312.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,278581.0,1.0,NaN,NaN,NaN,1.0,LAR3,12447.0,268548.0,96451.0,NaN,1.0,NaN,NaN,1964.0,NaN,NaN,73681.0,119407.0,45726.0,1533.89,NaN,NaN,6.037124e+13,Central,NaN,NaN,Central,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77376,NaN,NaN,2.0,NaN,NaN,NaN,NaN,6.0,2.0,NaN,NaN,1286.0,1286.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,47405.0,NaN,NaN,NaN,NaN,NaN,LARE40,12447.0,27328.0,96284.0,NaN,1.0,NaN,NaN,1940.0,NaN,NaN,70917.0,354621.0,283704.0,4478.43,NaN,NaN,6.037101e+13,NaN,NaN,NaN,Central,NaN,NaN
77377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1612.0,1612.0,1612.0,NaN,NaN,1612.0,NaN,1.0,2.0,2.0,560.0,NaN,12105.0,NaN,NaN,NaN,NaN,NaN,NaN,27110.0,NaN,97116.0,NaN,NaN,656.0,NaN,1964.0,1.0,NaN,50683.0,67205.0,16522.0,1107.48,NaN,NaN,6.111008e+13,NaN,NaN,NaN,NaN,NaN,NaN
77378,NaN,NaN,2.0,NaN,NaN,1.0,NaN,4.0,1.0,NaN,NaN,1032.0,1032.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,5074.0,NaN,NaN,NaN,NaN,NaN,SER1*,36502.0,NaN,96480.0,NaN,1.0,NaN,NaN,1954.0,NaN,NaN,32797.0,49546.0,16749.0,876.43,NaN,NaN,6.037434e+13,Central,NaN,NaN,Central,NaN,NaN
77379,NaN,NaN,2.0,NaN,NaN,NaN,NaN,6.0,2.0,NaN,NaN,1762.0,1762.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6347.0,NaN,NaN,NaN,NaN,NaN,WHR16000*,14634.0,NaN,96171.0,NaN,1.0,NaN,NaN,1955.0,NaN,NaN,140000.0,522000.0,382000.0,6317.15,NaN,NaN,6.037503e+13,NaN,NaN,NaN,Central,NaN,NaN


In [103]:
df.isna().sum(axis = 1) / df.shape[0]

0        0.000465
1        0.000426
2        0.000439
3        0.000414
4        0.000375
           ...   
77376    0.000426
77377    0.000414
77378    0.000414
77379    0.000439
77380    0.000439
Length: 77381, dtype: float64

In [ ]:
# I still think I should be able to use what I've already done above with value_counts to complete this.

In [113]:
df.isnull()/df.shape[1]

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,...,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,num_cols_missing,pct_cols_missing
0,0.0,0.013889,0.013889,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.0,0.0,0.013889,0.0,0.013889,0.013889,0.0,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.0,0.000000,0.000000,0.013889,0.0,0.0,0.0,0.013889,0.013889,0.013889,0.013889,...,0.013889,0.0,0.0,0.0,0.013889,0.0,0.0,0.000000,0.013889,0.013889,0.013889,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.013889,0.0,0.0
1,0.0,0.013889,0.013889,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.0,0.0,0.013889,0.0,0.013889,0.000000,0.0,0.0,0.013889,0.013889,0.000000,0.013889,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,0.0,0.013889,0.013889,0.013889,0.013889,...,0.013889,0.0,0.0,0.0,0.013889,0.0,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.000000,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.013889,0.0,0.0
2,0.0,0.013889,0.013889,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.0,0.0,0.013889,0.0,0.013889,0.013889,0.0,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.0,0.000000,0.000000,0.013889,0.0,0.0,0.0,0.000000,0.013889,0.013889,0.013889,...,0.013889,0.0,0.0,0.0,0.013889,0.0,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.000000,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.013889,0.0,0.0
3,0.0,0.013889,0.013889,0.0,0.000000,0.013889,0.013889,0.013889,0.0,0.013889,0.0,0.0,0.000000,0.0,0.013889,0.013889,0.0,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.0,0.013889,0.013889,0.013889,0.0,0.0,0.0,0.000000,0.013889,0.013889,0.013889,...,0.000000,0.0,0.0,0.0,0.013889,0.0,0.0,0.013889,0.000000,0.013889,0.013889,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.013889,0.000000,0.0,0.013889,0.013889,0.0,0.0
4,0.0,0.013889,0.013889,0.0,0.000000,0.013889,0.013889,0.000000,0.0,0.013889,0.0,0.0,0.000000,0.0,0.013889,0.013889,0.0,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.0,0.013889,0.013889,0.013889,0.0,0.0,0.0,0.000000,0.013889,0.013889,0.013889,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.013889,0.000000,0.013889,0.013889,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.000000,0.013889,0.013889,0.000000,0.0,0.013889,0.013889,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77376,0.0,0.013889,0.013889,0.0,0.000000,0.013889,0.013889,0.013889,0.0,0.013889,0.0,0.0,0.000000,0.0,0.013889,0.013889,0.0,0.0,0.013889,0.013889,0.013889,0.013889,0.0,0.013889,0.0,0.013889,0.013889,0.013889,0.0

In [193]:
df_col = df

In [196]:
num_missing = df.isnull().sum()
rows_count = df.shape[0]
pct_missing = num_missing / rows_count

In [197]:
cols_missing = pd.DataFrame({'number_missing_rows': num_missing, 'pct_rows_missing': pct_missing})

In [200]:
# Ryan's solutions:

def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows_count = df.shape[0]
    pct_missing = num_missing / rows_count
    cols_missing = pd.DataFrame({'number_missing_rows': num_missing, 'pct_rows_missing': pct_missing})
    return cols_missing

In [199]:
nulls_by_col(df)

,number_missing_rows,pct_rows_missing
parcelid,0,0.000000
typeconstructiontypeid,77159,0.997131
storytypeid,77331,0.999354
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,27941,0.361083
buildingclasstypeid,77366,0.999806
architecturalstyletypeid,77175,0.997338
airconditioningtypeid,52428,0.677531
id,0,0.000000
basementsqft,77331,0.999354


In [201]:
# Summary; I need to better understand how the [] and [[]] work together to creating iteration and when they do not create iteration.

# I still think there's a way to use value_counts() to get this done, but I'm running into problems with the %

# So I was close; Ryan in his answers kinda used that concept.

In [180]:
# Attempting to build dataframe only using .value_counts to group.
# (df.isna().sum(axis = 1).value_counts(normalize = True).sort_index() * 100).reset_index()

df3_test = pd.DataFrame(df.isna().sum(axis = 1).value_counts().sort_index().reset_index())

In [181]:
df3_test.head()

,index,0
0,23,2
1,24,13
2,25,24
3,26,65
4,27,316


In [171]:
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,...,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,num_cols_missing
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,3.5,4.0,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,...,122,NaN,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13,0,0.025595,14297519,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,36
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,1.0,2.0,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,...,1110,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13,1,0.055619,17052889,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,33
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,...,122,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13,2,0.005383,14186244,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,34
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,...,0101,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13,3,-0.103410,12177905,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN,32
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,3.0,3.0,8.0,3.0,NaN,NaN,1312.0,1312.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34185120.0,-118414640.0,278581.0,1.0,NaN,NaN,NaN,...,010C,LAR3,6.037124e+07,12447.0,3101.0,268548.0,96451.0,0.0,NaN,1.0,NaN,NaN,1964.0,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,NaN,NaN,6.037124e+13,4,0.006940,10887214,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN,29


In [202]:
# Ryan's solution:

def nulls_by_row(df):
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
    rows_missing = pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing}).reset_index().groupby(['num_cols_missing','pct_cols_missing']).count().rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing 

null_rows = nulls_by_row(df)
null_rows.sort_values(by="pct_cols_missing", ascending=False, inplace=True)
null_rows

,num_cols_missing,pct_cols_missing,num_rows
25,48,67.6056338028169,3
24,47,66.19718309859155,3
23,46,64.7887323943662,5
22,45,63.38028169014085,50
21,44,61.97183098591549,78
20,43,60.56338028169014,28
19,42,59.154929577464785,23
18,41,57.74647887323944,29
17,40,56.33802816901409,230
16,39,54.929577464788736,285


In [203]:
# Nice. So the key here is that I use the .value_counts AFTER we create the function. Not inside the function.

# Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [29]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 70)

In [187]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 71 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77381 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77381 non-null  float64
 4   heatingorsystemtypeid         49440 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      206 non-null    float64
 7   airconditioningtypeid         24953 non-null  float64
 8   id                            77381 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77381 non-null  float64
 11  bedroomcnt                    77381 non-null  float64
 12  buildingqualitytypeid         49672 non-null  float64
 13  c

In [26]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52320
Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

#### Plan for Isolating Single Use properties:

Since the vast majority of all single use properties appear to be single use homes, I'm going to use the `propertylandusetypeid` == 261, to filter down to the Single Family Residential type properties. That way I'm not only using `unitcnt`.

In [191]:
# Removing all properties except single family residential

df[df.propertylandusetypeid == 261]

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,...,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,num_cols_missing
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,3.5,4.0,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,...,122,NaN,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13,0,0.025595,14297519,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,36
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,1.0,2.0,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,...,1110,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13,1,0.055619,17052889,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,33
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,...,122,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13,2,0.005383,14186244,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,34
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,...,0101,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13,3,-0.103410,12177905,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN,32
6,12095076,NaN,NaN,261.0,2.0,NaN,NaN,1.0,781532,NaN,3.0,4.0,9.0,3.0,NaN,NaN,2962.0,2962.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34145202.0,-118179824.0,63000.0,1.0,NaN,NaN,NaN,...,0101,PSR2,6.037461e+07,47019.0,3101.0,274684.0,96293.0,0.0,NaN,1.0,NaN,NaN,1950.0,NaN,NaN,276684.0,773303.0,2016.0,496619.0,9516.26,NaN,NaN,6.037461e+13,6,-0.001011,12095076,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77376,11000655,NaN,NaN,261.0,2.0,NaN,NaN,NaN,673515,NaN,2.0,2.0,6.0,2.0,NaN,NaN,1286.0,1286.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,34245368.0,-118282383.0,47405.0,NaN,NaN,NaN,NaN,...,0100,LARE40,6.037101e+07,12447.0,3101.0,27328.0,96284.0,0.0,NaN,1.0,NaN,NaN,1940.0,NaN,NaN,70917.0,354621.0,2016.0,283704.0,4478.43,NaN,NaN,6.037101e+13,77609,0.020615,11000655,2017-09-20,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN,33
77377,17239384,NaN,NaN,261.0,NaN,

In [208]:
# Looking at the type of building and remove those that aren't highly likely to be single unit properties...

include_ids = [261.0, 260.0, 262.0, 263.0, 264.0]
df = df[df.propertylandusetypeid.isin(include_ids)]
df.shape

(52437, 71)

In [209]:
df.unitcnt.value_counts()

1.0    33803
2.0       28
4.0        2
3.0        1
Name: unitcnt, dtype: int64

## Question 2:

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

- The input:
    - A dataframe
    - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
    - The dataframe with the columns and rows dropped as indicated. *Be sure to drop the columns prior to the rows in your function.*
- **Hint:**
    - Look up the dropna documentation.
    - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
    - Make use of inplace, i.e. inplace=True/False.